Code for scraping from indeed.co.uk

In [111]:
#import packages

import requests, bs4, time
import pandas as pd


In [112]:
#define functions for parsing HTML

def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

def extract_salary_from_result(soup): 
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span",attrs={"class":"salaryText"}).text)
        except:
            salaries.append("Nothing_found")
    return(salaries)

def extract_location_from_result(soup): 
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find("span", attrs={"class": "location accessible-contrast-color-location"}).text)
        except:
            locations.append("Nothing_found")
   
    return(locations)

def extract_description_from_result(soup): 
    description = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            description.append(div.find("div", attrs={"class": "summary"}).text)
        except:
            description.append("Nothing_found")
   
    return(description)

def extract_date_from_result(soup): 
    date = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            date.append(div.find("span", attrs={"class": "date"}).text)
        except:
            date.append("Nothing_found")
   
    return(date)

def extract_company_from_result(soup): 
    company = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            company.append(div.find("span", attrs={"class": "company"}).text)
        except:
            company.append("Nothing_found")
   
    return(company)

def extract_links(soup):
    links =[]
    for div in soup.find_all(name='a', attrs={'class':'jobtitle turnstileLink'}):
        links.append('https://www.indeed.co.uk'+str(div['href']))
    return links

def extract_full_desc(soup):
    text=[x.text for x in soup.find_all(name="div",attrs={"id":"jobDescriptionText"})]
    return text


def extract_headlines_from_result(soup): 
    headlines = pd.DataFrame(columns="location","type","salary")
    list=[x.text for x in soup.find_all(name="span",attrs={"class":"jobsearch-JobMetadataHeader-iconLabel"})]
    try:
        headlines['location']
    
        except:
            salaries.append("Nothing_found")
    return(salaries)


In [121]:
#scraping code:

#decide what search term to use for finding jobs
searchTerm="data"

#create empty data frame with column headers
ads=pd.DataFrame(columns=['company','job_title','salary','location','description','date','full_description','other_deets'])

# loop for scraping

for i in range(0,1000,10):
    time.sleep(1) #ensuring at least 1 second between page grabs
    url = "https://www.indeed.co.uk/jobs?q="+searchTerm+"&filter=0&l="+"&start="+str(i)
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.content)
    df=pd.DataFrame()
    df['company']=extract_company_from_result(soup)
    df['job_title']=extract_job_title_from_result(soup)
    df['salary']=extract_salary_from_result(soup)
    df['location']=extract_location_from_result(soup)
    df['description']=extract_description_from_result(soup)
    df['date']=extract_date_from_result(soup)
    
    sub_urls=extract_links(soup)
    text_list=[]
    deets_list=[]
    for j in sub_urls:
        res_sub = requests.get(j)
        soup_sub = bs4.BeautifulSoup(res_sub.content)
        desc=extract_full_desc(soup_sub)
        text_list.append(desc)
        other_deets=extract_headlines_from_result(soup_sub)
        deets_list.append(other_deets)
    df['full_description']=text_list
    df['other_deets']=other_deets
    ads=ads.append(df)
    

In [130]:
#establish how many ads had no salary
ads[ads['salary']=="Nothing_found"].shape

(704, 8)

In [249]:
#print data to csv
#ads.to_csv(r"data\indeed_scrape_data.csv")